Bathymetry Transformer

In [4]:
import numpy as np
import xarray as xr
import os
import re

def convert_netcdf_to_grd(nc_file, grd_file, x_var='longitude', y_var='latitude', z_var='elevation'):

    try:
        with xr.open_dataset(nc_file) as ds:
            x = ds[x_var].values
            y = ds[y_var].values
            z = ds[z_var].values

            if y[0] > y[-1]:
                y = y[::-1]
                z = np.flipud(z)

            ncols = len(x)
            nrows = len(y)
            xmin, xmax = np.min(x), np.max(x)
            ymin, ymax = np.min(y), np.max(y)
            zmin, zmax = np.min(z), np.max(z)

            with open(grd_file, 'w') as f:
                f.write('DSAA\n')
                f.write(f'{ncols} {nrows}\n')
                f.write(f'{xmin:.8f} {xmax:.8f}\n')
                f.write(f'{ymin:.8f} {ymax:.8f}\n')
                f.write(f'{zmin:.8f} {zmax:.8f}\n')
                
                for i in range(nrows):
                    row_str = ' '.join(f'{val:.8f}' for val in z[i, :])
                    f.write(row_str + '\n')

        print(f"✅ Convertido: {os.path.basename(nc_file)} → {os.path.basename(grd_file)}")

    except FileNotFoundError:
        print(f"❌ No se encontró: {nc_file}")
    except KeyError as e:
        print(f"❌ Variable no encontrada ({e}). Revisa nombres {x_var}, {y_var}, {z_var}.")
    except Exception as e:
        print(f"⚠️ Error inesperado con {nc_file}: {e}")

if __name__ == "__main__":
    carpeta_entrada = os.path.join(r"Datos/HySEA/GrdNetCDF")
    carpeta_salida  = os.path.join(r"Datos/EasyWave/GrdASCII")

    os.makedirs(carpeta_salida, exist_ok=True)

    patron = re.compile(r"grilla_NetCDF_(\d+)\.grd$")
    archivos = [f for f in os.listdir(carpeta_entrada) if patron.match(f)]

    if not archivos:
        print("⚠️ No se encontraron archivos grilla_NetCDF_{n}.grd en la carpeta de entrada.")
    else:
        archivos.sort(key=lambda f: int(patron.match(f).group(1)))

        for archivo in archivos:
            numero = patron.match(archivo).group(1)
            input_file = os.path.join(carpeta_entrada, archivo)
            output_file = os.path.join(carpeta_salida, f"salida_ascii_{numero}.grd")

            convert_netcdf_to_grd(input_file, output_file, x_var='x', y_var='y', z_var='z')


✅ Convertido: grilla_NetCDF_1.grd → salida_ascii_1.grd


Parameters Transformer:

In [5]:
import os
import re

def convertir_faults_a_flt(input_txt, output_flt):
    with open(input_txt, 'r') as fin, open(output_flt, 'w') as fout:
        next(fin)

        for line in fin:
            valores = line.strip().split()
            if not valores:
                continue

            valores = valores[1:]
            valores = [f"{float(v):.3f}" for v in valores]

            salida = (
                f"-location {valores[0]} {valores[1]} {valores[2]} "
                f"-size {valores[3]} {valores[4]} "
                f"-strike {valores[5]} "
                f"-dip {valores[6]} "
                f"-rake {valores[7]} "
                f"-slip {valores[8]}"
            )

            fout.write(salida + "\n")

    print(f"✅ Convertido: {os.path.basename(input_txt)} → {os.path.basename(output_flt)}")


if __name__ == "__main__":
    
    carpeta_entrada = r"Datos/HySEA/ParametersTXT"
    carpeta_salida  = r"Datos/EasyWave/ParametersFLT"

    os.makedirs(carpeta_salida, exist_ok=True)

    patron = re.compile(r"data_parameters_(\d+)\.txt$")
    archivos = [f for f in os.listdir(carpeta_entrada) if patron.match(f)]

    if not archivos:
        print("⚠️ No se encontraron archivos data_parameters_{n}.txt en la carpeta Hysea.")
    else:
        archivos.sort(key=lambda f: int(patron.match(f).group(1)))

        for archivo in archivos:
            numero = patron.match(archivo).group(1)
            input_file = os.path.join(carpeta_entrada, archivo)
            output_file = os.path.join(carpeta_salida, f"data_parameters_{numero}.flt")

            convertir_faults_a_flt(input_file, output_file)


✅ Convertido: data_parameters_1.txt → data_parameters_1.flt
